In [1]:
from keras.models import load_model
import numpy as np

Using TensorFlow backend.


In [2]:
MODEL_NAME = './models/baseline_model.hdf5'
AUDIO_INPUT = './data/test_inputs/X_test_audio1096.npy'
GESTURE_OUTPUT = './data/test_outputs/predict_1096.txt'

In [3]:
model = load_model(MODEL_NAME)

In [4]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_4 (TimeDist (None, 61, 256)           6912      
_________________________________________________________________
batch_normalization_5 (Batch (None, 61, 256)           1024      
_________________________________________________________________
activation_6 (Activation)    (None, 61, 256)           0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 61, 256)           0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 61, 256)           65792     
_________________________________________________________________
batch_normalization_6 (Batch (None, 61, 256)           1024      
_________________________________________________________________
activation_7 (Activation)    (None, 61, 256)          

In [5]:
X = np.load(AUDIO_INPUT)

In [6]:
X.shape

(205, 61, 26)

In [7]:
pred = model.predict(X)

In [8]:
pred.shape

(205, 384)

Dimension of the predicted data is `(243, 384)`, where 243 denotes the number of time steps, and 384 denotes the x,y,z positions and velocities of the 64 joints (64*(3+3) = 384). Because of that we first need to remove the velocities.

In [9]:
def remove_velocity(data, dim=3):
    """Remove velocity values from raw prediction data

      Args:
          data:         array containing both position and velocity values
          dim:          gesture dimensionality

      Returns:
          np.ndarray:   array containing only position values
    """

    starts = np.arange(0, data.shape[1], dim * 2)
    stops = np.arange(dim, data.shape[1], dim * 2)
    return np.hstack([data[:, i:j] for i, j in zip(starts, stops)])

In [10]:
# Remove velocity values
only_pos = remove_velocity(pred)

In [11]:
only_pos.shape

(205, 192)

We can see that we now have the dimension of the data `(243, 192)`

In [12]:
np.savetxt(GESTURE_OUTPUT, only_pos)